In [ ]:
import pandas as pd
from graphdatascience import GraphDataScience
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

gds = GraphDataScience('bolt://localhost:7687', auth = ('neo4j', 'wxp614503'))

In [1]:
# Clear last graph - All data and schema attributes
gds.run_cypher('MATCH(n) DETACH DELETE n')
gds.run_cypher('CALL apoc.schema.assert({},{})')

NameError: name 'gds' is not defined

In [27]:
gds.run_cypher('CREATE CONSTRAINT job_unique IF NOT EXISTS ON (n:Job) ASSERT n.Id IS UNIQUE')
gds.run_cypher('CREATE CONSTRAINT freelancer_unique IF NOT EXISTS ON (n:Freelancer) ASSERT n.Id IS UNIQUE')

gds.run_cypher('CREATE CONSTRAINT area_unique IF NOT EXISTS ON (n:Area) ASSERT n.Name IS UNIQUE')
gds.run_cypher('CREATE CONSTRAINT skill_unique IF NOT EXISTS ON (n:Skill) ASSERT n.Name IS UNIQUE')
gds.run_cypher('CREATE CONSTRAINT certification_unique IF NOT EXISTS ON (n:Certification) ASSERT n.Name IS UNIQUE')

""


In [42]:
folder = 'C:/Users/xwang338/Desktop/Project/2022 - US - Talent Exchange NLP/Data/'
file = 'freelancers.xlsx'

df = pd.read_excel(folder + file, sheet_name = 'certification')

In [52]:
import re

df = df.dropna(subset = ['CertificationValue'])
df['CertificationValue'] = df['CertificationValue'].str.replace(re.compile('[^a-zA-Z0-9 \n\.\+\#\(\)]'), '')



In [53]:
df.to_csv(folder + 'freelancers_certification.csv', index = False)

# Job

In [28]:
gds.run_cypher(
    '''
    LOAD CSV WITH HEADERS FROM 'file:///jobs_summary.csv' AS line
    MERGE (n:Job {Id: line.jobid}) SET n.Title = line.jobtitle
    '''
)

""


In [29]:
gds.run_cypher(
    '''
    LOAD CSV WITH HEADERS FROM 'file:///jobs_focus_area.csv' AS line
    MATCH (n:Job) WHERE n.Id = line.jobid
    MERGE (m:Area {Name: line.FocusAreaDescription})
    MERGE (n)-[r:FOCUS]->(m)
    '''
)

""


In [30]:
gds.run_cypher(
    '''
    LOAD CSV WITH HEADERS FROM 'file:///jobs_skill.csv' AS line
    MATCH (n:Job) WHERE n.Id = line.jobid
    MERGE (m:Skill {Name: line.skillname})
    MERGE (n)-[r:REQUIRE]->(m)
    '''
)

""


In [31]:
gds.run_cypher(
    '''
    LOAD CSV WITH HEADERS FROM 'file:///jobs_certification.csv' AS line
    MATCH (n:Job) WHERE n.Id = line.jobid
    MERGE (m:Certification {Name: line.CertificationValue})
    MERGE (n)-[r:REQUIRE]->(m)
    '''
)

""


# Freelancer

In [32]:
gds.run_cypher(
    '''
    LOAD CSV WITH HEADERS FROM 'file:///freelancers_summary.csv' AS line
    MERGE (n:Freelancer {Id: line.FreelancerID})
    '''
)

""


In [33]:
gds.run_cypher(
    '''
    LOAD CSV WITH HEADERS FROM 'file:///freelancers_focus_area.csv' AS line
    MATCH (n:Freelancer) WHERE n.Id = line.FreelancerID
    MERGE (m:Area {Name: line.FocusAreaDescription})
    MERGE (n)-[r:FOCUS]->(m)
    '''
)

""


In [55]:
gds.run_cypher(
    '''
    LOAD CSV WITH HEADERS FROM 'file:///freelancers_certification.csv' AS line
    WITH line where line.CertificationValue IS NOT NULL
    MATCH (n:Freelancer) WHERE n.Id = line.FreelancerID
    MERGE (m:Certification {Name: line.CertificationValue})
    MERGE (n)-[r:HAS]->(m)
    '''
)

""


# Job & Freelancer

In [56]:
gds.run_cypher(
    '''
    LOAD CSV WITH HEADERS FROM 'file:///shortlist.csv' AS line
    MATCH (n:Freelancer) WHERE n.Id = line.FreelancerID
    MATCH (m:Job) WHERE m.Id = line.jobid
    MERGE (n)-[r:MATCH]->(m)
    '''
)

""


In [57]:
gds.run_cypher(
    '''
    LOAD CSV WITH HEADERS FROM 'file:///active_engagement.csv' AS line
    MATCH (n:Freelancer) WHERE n.Id = line.FreelancerID
    MATCH (m:Job) WHERE m.Id = line.jobid
    MERGE (n)-[r:MATCH]->(m)
    '''
)

""


# Search

In [3]:
pd.set_option('display.max_colwidth', None)
model = SentenceTransformer('C:/Users/xwang338/Desktop/Project/2022 - US - Talent Exchange NLP/Model')

### Skill to Skill

In [22]:
name = '.NET'

cypher = '''
MATCH (n1:Skill)-[r1]-(n2)-[r2]-(n3:Skill)
WHERE n1.Name = '{}'
RETURN distinct(n3.Name) AS Result
'''.format(name)

df = gds.run_cypher(cypher)

sentences = df['Result'].values.tolist()
sentences.insert(0, name)

embeddings = model.encode(sentences)

similarity = cosine_similarity([embeddings[0]], embeddings[1:])
df['Similarity'] = similarity[0]
df.sort_values('Similarity', ascending = False)[1:11]

,Result,Similarity
4098,MS .NET,0.828114
768,NET,0.788151
3769,.NET 3.5,0.778005
2582,.NET 4.5,0.759161
1982,.NET 2.0,0.747829
4794,.NET 3.0,0.744971
314,C#.NET,0.741844
36,ASP.NET,0.711043
2116,.NET FRAMEWORK,0.685938
6259,NET DEVELOPMENT,0.669760


### Skill to Certification

In [27]:
df = gds.run_cypher(
    '''
    MATCH (n1:Skill)-[r1]-(n2)-[r2]-(n3:Certification)
    WHERE n1.Name = 'MORTGAGE OPERATIONS'
    RETURN distinct(n3.Name) AS Result
    '''
)

sentences = df['Result'].values.tolist()
sentences.insert(0, 'MORTGAGE OPERATIONS')

embeddings = model.encode(sentences)

similarity = cosine_similarity([embeddings[0]], embeddings[1:])
df['Similarity'] = similarity[0]
df.sort_values('Similarity', ascending = False)[1:21]

,Result,Similarity
1,Licensed Realtor Keller Williams (State of FL),0.259704
3,Licensed Insurance Adjuster All Lines,0.210485
6,CPA,0.204352
0,Certified Regulatory Compliance Manager,0.184436
4,Public Notary,0.176066
5,Life and health insurance,0.147559
7,Certified Public Accountant (CPA) Actively Licensed in the Commonwealth of Virginia,0.127877
2,Florida Notary Public (Remote Online Notary Certified Mobile Signing Agent),0.118595


### Certification to Skill

In [7]:
df = gds.run_cypher(
    '''
    MATCH (n1:Certification)-[r1]-(n2)-[r2]-(n3:Skill)
    WHERE n1.Name = 'Microsoft Certified Azure Fundamentals'
    RETURN distinct(n3.Name) AS Result
    '''
)

sentences = df['Result'].values.tolist()
sentences.insert(0, 'Microsoft Certified Azure Fundamentals')

embeddings = model.encode(sentences)

similarity = cosine_similarity([embeddings[0]], embeddings[1:])
df['Similarity'] = similarity[0]
df.sort_values('Similarity', ascending = False)[1:21]

,Result,Similarity
126,MICROSOFT CLOUD - AZURE,0.682908
48,AZURE,0.671717
186,MICROSOFT AZURE SQL,0.576094
59,MICROSOFT,0.452627
98,AWS CERTIFIED,0.451269
209,.NET CORE,0.390944
72,CLOUD SERVICES,0.384945
62,MICROSOFT OFFICE,0.374246
117,MICROSOFT OFFICE SUITE,0.372953
142,VIRTUALIZATION INFRASTRUCTURE,0.345281


### Certification to Certification

In [8]:
df = gds.run_cypher(
    '''
    MATCH (n1:Certification)-[r1]-(n2)-[r2]-(n3:Certification)
    WHERE n1.Name = 'Microsoft Certified Azure Fundamentals'
    RETURN distinct(n3.Name) AS Result
    '''
)

sentences = df['Result'].values.tolist()
sentences.insert(0, 'Microsoft Certified Azure Fundamentals')

embeddings = model.encode(sentences)

similarity = cosine_similarity([embeddings[0]], embeddings[1:])
df['Similarity'] = similarity[0]
df.sort_values('Similarity', ascending = False)[1:21]

,Result,Similarity
0,Microsoft Certified Azure AI Fundamentals,0.809052
41,Microsoft Certified Azure Solutions Architect Expert,0.763384
39,Microsoft Certified Azure Administrator Associate,0.729151
16,Microsoft Certified,0.708583
26,Microsoft Certified Azure Security Engineer Associate,0.704610
42,Microsoft Cloud (Azure),0.685291
97,Microsoft Certified Professional,0.678548
74,Azure Databrick Certified,0.625291
35,Microsoft Certified DevOps Engineer Expert,0.598412
103,Microsoft Certified Technology Specialist ( MCTS ) .Net Framework 2.0 Web Applications,0.553795


### Area to Skill

In [8]:
df = gds.run_cypher(
    '''
    MATCH (n1:Area)-[r1]-(n2)-[r2]-(n3:Skill)
    WHERE n1.Name = 'Finance Operations'
    RETURN distinct(n3.Name) AS Result
    '''
)

sentences = df['Result'].values.tolist()
sentences.insert(0, 'Finance Operations')

embeddings = model.encode(sentences)

similarity = cosine_similarity([embeddings[0]], embeddings[1:])
df['Similarity'] = similarity[0]
df.sort_values('Similarity', ascending = False)[1:21]

,Result,Similarity
1397,FINANCIAL OPERATIONS,0.939034
837,OPERATIONS FINANCE,0.917218
8033,FINANCE / ACCOUNTING OPERATIONS,0.874775
8402,OTHER FINANCIAL OPERATIONS,0.860975
9207,ACCOUNTING & FINANCE OPERATIONS,0.857165
5490,FINANCE MANAGEMENT,0.815904
450,Finance,0.801314
419,FINANCE,0.801314
12012,OPERATIONAL FINANCE,0.797847
14834,SALES OPERATIONS FINANCE,0.793662


### Area to Certification

In [16]:
df = gds.run_cypher(
    '''
    MATCH (n1:Area)-[r1]-(n2)-[r2]-(n3:Certification)
    WHERE n1.Name = 'Finance Operations'
    RETURN distinct(n3.Name) AS Result
    '''
)

sentences = df['Result'].values.tolist()
sentences.insert(0, 'Finance Operations')

embeddings = model.encode(sentences)

similarity = cosine_similarity([embeddings[0]], embeddings[1:])
df['Similarity'] = similarity[0]
df.sort_values('Similarity', ascending = False)[1:21]

,Result,Similarity
1246,Finance and Insurance,0.663499
1587,Securities Operations Overview I,0.662800
6620,Corporate Finance Accounting,0.656432
4988,Business of Shipping Financial Management,0.638856
546,Finance Training Society,0.619971
7282,Emerging Issues in Operations,0.618675
2490,Finance Risk Manager,0.616463
3786,Financial services specialist,0.616402
666,Financial Industry Regulatory Authority,0.611522
1245,Accounting,0.607816


### Skill to Area

In [19]:
df = gds.run_cypher(
    '''
    MATCH (n1:Skill)-[r1]-(n2)-[r2]-(n3:Area)
    WHERE n1.Name = 'ACCESS CONTROL'
    RETURN distinct(n3.Name) AS Result
    '''
)

sentences = df['Result'].values.tolist()
sentences.insert(0, 'ACCESS CONTROL')

embeddings = model.encode(sentences)

similarity = cosine_similarity([embeddings[0]], embeddings[1:])
df['Similarity'] = similarity[0]
df.sort_values('Similarity', ascending = False)[1:21]

,Result,Similarity
0,Change Management,0.369716
9,Operations,0.349080
17,Audit,0.308890
3,Application Technology,0.306323
20,Finance Operations,0.298261
10,User Experience,0.279011
7,SAP,0.273189
19,Cloud Computing and Networking,0.269428
18,Data and Analytics,0.266607
2,"Cybersecurity, Privacy & Forensics",0.258913


In [27]:
data = pd.read_excel('C:/Users/xwang338/Desktop/Project/2022 - US - Talent Exchange NLP/Data/New Microsoft Excel Worksheet.xlsx')
skill_list = data['Skill_Name'].tolist()

In [28]:
fnl_dic = {}

for name in skill_list:

    try:
        cypher = '''
        MATCH (n1:Skill)-[r1]-(n2)-[r2]-(n3:Skill)
        WHERE n1.Name = '{}'
        RETURN distinct(n3.Name) AS Result
        '''.format(name)

        df = gds.run_cypher(cypher)

        sentences = df['Result'].values.tolist()
        sentences.insert(0, name)

        embeddings = model.encode(sentences)

        similarity = cosine_similarity([embeddings[0]], embeddings[1:])
        df['Similarity'] = similarity[0]
        df = df.sort_values('Similarity', ascending = False)

        fnl_dic[name] = df['Result'][1:11].tolist()
    
    except:
        pass


In [26]:
fnl_dic

{'.NET': ['MS .NET',
  'NET',
  '.NET 3.5',
  '.NET 4.5',
  '.NET 2.0',
  '.NET 3.0',
  'C#.NET',
  'ASP.NET',
  '.NET FRAMEWORK',
  'NET DEVELOPMENT'],
 'ABAP FOR HANA': ['ABAP',
  'SAP HANA',
  'ABAP  SD',
  'S/4 HANA',
  'SAP HANA DB',
  'ABAP/4',
  'HANA CLOUD PLATFORM',
  'SAP HANA STUDIO',
  'BI S4/HANA',
  'ABAP DEVELOPER'],
 'ABINITIO': ['Ab Initio',
  'ALGO',
  'IAAS',
  'TRANSLATE',
  'BO',
  'TRANSLATED',
  'RELTIO',
  'OBIEE/OBIA',
  'TIBCO',
  'ABAP'],
 'ADF': ['ACTIVE DIRECTORY',
  'FRAMEWORK',
  'MICROSOFT ACTIVE DIRECTORY',
  'MFA',
  'DIGITAL ADVERTISING',
  'EXCHANGE',
  'EXCHANGE 2000',
  'SALESFORCE',
  'ASP',
  'FA'],
 'Adobe AEP': ['ADOBE CREATIVE CLOUD',
  'ADOBE DOCUMENT CLOUD',
  'ADOBE ILLUSTRATOR',
  'AGILE, SCRUM (CSP, CSPO)',
  'MICROSOFT 365',
  'FINANCIAL RISK MANAGEMENT',
  'INTERNET OF THINGS',
  'PROGRAM MANAGEMENT',
  'APPLICATION ARCHITECTURE',
  'CLOUD SECURITY']}

In [29]:
fnl_df = pd.DataFrame.from_dict(fnl_dic, orient='index')

In [30]:
fnl_df

,0,1,2,3,4,5,6,7,8,9
.NET,MS .NET,NET,.NET 3.5,.NET 4.5,.NET 2.0,.NET 3.0,C#.NET,ASP.NET,.NET FRAMEWORK,NET DEVELOPMENT
ABAP FOR HANA,ABAP,SAP HANA,ABAP SD,S/4 HANA,SAP HANA DB,ABAP/4,HANA CLOUD PLATFORM,SAP HANA STUDIO,BI S4/HANA,ABAP DEVELOPER
ABINITIO,Ab Initio,ALGO,IAAS,TRANSLATE,BO,TRANSLATED,RELTIO,OBIEE/OBIA,TIBCO,ABAP
ADF,ACTIVE DIRECTORY,FRAMEWORK,MICROSOFT ACTIVE DIRECTORY,MFA,DIGITAL ADVERTISING,EXCHANGE,EXCHANGE 2000,SALESFORCE,ASP,FA
Adobe AEP,ADOBE CREATIVE CLOUD,ADOBE DOCUMENT CLOUD,ADOBE ILLUSTRATOR,"AGILE, SCRUM (CSP, CSPO)",MICROSOFT 365,FINANCIAL RISK MANAGEMENT,INTERNET OF THINGS,PROGRAM MANAGEMENT,APPLICATION ARCHITECTURE,CLOUD SECURITY
...,...,...,...,...,...,...,...,...,...,...
WEBCENTER,WEB PORTAL,ORACLE ADF WEBCENTER,WEBTRENDS,CUSTOMER DATA HUB,ORACLE WEB CENTER IMAGING,NETSUITE,MARKETING CLOUD,SHAREPOINT,IBM WEB SPHERE PORTAL,HTTP
WEBLOGIC,WEBLOGIC SERVER,WEBLOGIC PORTAL,ORACLE WEBLOGIC,WEBLOGIC WS,WEBLOGIC SERVERS,WEBSHERE/WEBLOGIC,WEBLOGIC WORKBENCH,BEA WEBLOGIC,WEBLOGIC APP SERVER,Oracle WebLogic Server
WHITE COLLAR CRIME INVESTIGATIONS,"WHITE COLLAR, CORPORATE CRIME",CRIMINAL INVESTIGATIONS,CIVIL INVESTIGATIONS,GOVERNMENT INVESTIGATIONS,PRIVATE INVESTIGATIONS,INVESTIGATIONS,Investigations,FINANCIAL INVESTIGATIONS,CRIME,FRAUD INVESTIGATIONS
WORK CLEARANCE MANAGEMENT,WORK CONTROL SUPERVISION,MAXIMO WORK MANAGEMENT,PAYROLL,ADMIN PERSONNEL RELATED TOMANUFACTURING,MANAGEMENT,HUMAN CAPITAL MANAGEMENT,HR,WORK CO-ORDINATION,ADMINISTRATIVE TASKS,WORKFLOW


In [31]:
fnl_df.to_csv('C:/Users/xwang338/Desktop/Project/2022 - US - Talent Exchange NLP/Data/result.csv')